In [61]:
import pandas as pd
import matplotlib.pyplot as plt
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

/Users/danukalakshan/Projects/review-analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import data

In [2]:
data = pd.read_csv("/Users/danukalakshan/Projects/review-analysis/data/trustpilot/task_rabbit.csv")
data.head(10)

,reviewer,date,text,stars
0,Loris,2025-12-02T12:23:08.000Z,NaN,5
1,"John, Morgan",2025-11-26T17:29:50.000Z,Norbert is great.,5
2,KM,2025-10-26T14:48:22.000Z,"Great experience. Very prompt, reliable and fast.",5
3,customer,2025-10-20T17:29:53.000Z,NaN,5
4,MR R E BRANT,2025-10-16T12:27:03.000Z,Paid Ikea to have wardrobes built via task rab...,1
5,Divya Pavan,2025-10-13T09:40:58.000Z,The person was so experienced and he did the e...,5
6,Stefano Andreozzi,2025-10-01T17:50:28.000Z,NaN,5
7,cliente,2025-09-15T21:48:19.000Z,I really like the job he did and he also can s...,5
8,cliente,2025-09-11T22:11:13.000Z,"Excellent experience. Very responsive, profes...",5
9,cliente,2025-09-05T12:50:01.000Z,"The copmany and idea is great , especially if ...",4


Data exploration

In [3]:
data.isnull().sum()

reviewer     0
date         0
text        49
stars        0
dtype: int64

In [4]:
data.dtypes

reviewer    object
date        object
text        object
stars        int64
dtype: object

In [5]:
review_counts = data.groupby("reviewer").size()
review_counts[review_counts > 1]

reviewer
Francesca     2
cliente      66
customer      3
dtype: int64

In [6]:
data[data.reviewer == "cliente"]

,reviewer,date,text,stars
7,cliente,2025-09-15T21:48:19.000Z,I really like the job he did and he also can s...,5
8,cliente,2025-09-11T22:11:13.000Z,"Excellent experience. Very responsive, profes...",5
9,cliente,2025-09-05T12:50:01.000Z,"The copmany and idea is great , especially if ...",4
10,cliente,2025-08-06T19:21:10.000Z,The gentleman who installed the product was gr...,5
15,cliente,2025-07-20T02:30:12.000Z,Excellent work,5
...,...,...,...,...
190,cliente,2023-10-23T15:26:19.000Z,fast and organised assembly. good job.,5
191,cliente,2023-10-22T21:15:26.000Z,"Vitor was perfect. Punctual, friendly, profes...",5
192,cliente,2023-10-20T16:04:01.000Z,NaN,5
197,cliente,2023-10-03T11:23:00.000Z,"Excellent service, showed up on time. Would us...",5


Data preprocessing

In [7]:
# drop rows with null values

df = data.dropna()
df.shape

(151, 4)

In [8]:
df[df.reviewer == "cliente"].shape

(40, 4)

In [9]:
# drop records of reviewers who has more than 5 reviews

review_counts = df.groupby("reviewer").size()
valid_reviewers = review_counts[review_counts <= 5].index
df = df[df["reviewer"].isin(valid_reviewers)]
df.shape

(111, 4)

In [10]:
# convert date column to datetime
df["datetime"] = pd.to_datetime(df["date"], utc=True)
df.drop(columns=['date'], inplace=True)

In [11]:
df.dtypes

reviewer                 object
text                     object
stars                     int64
datetime    datetime64[ns, UTC]
dtype: object

In [19]:
# create language column

def detect_language_safe(text):
    if not isinstance(text, str) or len(text.split()) < 5:
        return None
    try:
        return detect(text)
    except LangDetectException:
        return None

df["language"] = df["text"].apply(detect_language)
df.head()

,reviewer,text,stars,datetime,language
1,"John, Morgan",Norbert is great.,5,2025-11-26 17:29:50+00:00,en
2,KM,"Great experience. Very prompt, reliable and fast.",5,2025-10-26 14:48:22+00:00,en
4,MR R E BRANT,Paid Ikea to have wardrobes built via task rab...,1,2025-10-16 12:27:03+00:00,it
5,Divya Pavan,The person was so experienced and he did the e...,5,2025-10-13 09:40:58+00:00,en
11,Ilaria Rodella,Puntualità ed efficienza,4,2025-07-31 10:49:07+00:00,it


In [20]:
# translate to english

def translate_to_english(text, lang):
    translator = GoogleTranslator(source="auto", target="en")
    
    if not text or lang in (None, "en"):
        return text
    try:
        return translator.translate(text)
    except Exception:
        return None

df["text_en"] = df.apply(
    lambda row: translate_to_english(row["text"], row["language"]),
    axis=1
)

df.head()

,reviewer,text,stars,datetime,language,text_en
1,"John, Morgan",Norbert is great.,5,2025-11-26 17:29:50+00:00,en,Norbert is great.
2,KM,"Great experience. Very prompt, reliable and fast.",5,2025-10-26 14:48:22+00:00,en,"Great experience. Very prompt, reliable and fast."
4,MR R E BRANT,Paid Ikea to have wardrobes built via task rab...,1,2025-10-16 12:27:03+00:00,it,Paid Ikea to have wardrobes built via task rab...
5,Divya Pavan,The person was so experienced and he did the e...,5,2025-10-13 09:40:58+00:00,en,The person was so experienced and he did the e...
11,Ilaria Rodella,Puntualità ed efficienza,4,2025-07-31 10:49:07+00:00,it,Punctuality and efficiency


In [43]:
def detect_language_safe(text):
    if not isinstance(text, str) or len(text.split()) < 5:
        return None
    try:
        return detect(text)
    except LangDetectException:
        return None

def translate_to_english(text, lang):
    translator = GoogleTranslator(source="auto", target="en")
    
    if not text or lang in (None, "en"):
        return text
    try:
        return translator.translate(text)
    except Exception:
        return None

def preprocess_pipeline(data_path, review_count_threshold=5):
    df = pd.read_csv(data_path)

    # schema check
    required_columns = {"reviewer", "date", "text", "stars"}
    missing = required_columns - set(df.columns)
    if missing:
        raise ValueError(f"Invalid data structure. Missing columns: {missing}")
    print("data preprocessing is started...\n")
        
    # drop rows with null values
    df.dropna(inplace=True)
    print("completed step 01: dropped rows with null values")

    # drop records of reviewers who has more than review_count_threshold reviews
    review_counts = df.groupby("reviewer").size()
    valid_reviewers = review_counts[review_counts <= review_count_threshold].index
    df = df[df["reviewer"].isin(valid_reviewers)]
    print(f"completed step 02: dropped records of reviewers who has more than {review_count_threshold} reviews")

    # convert date column to datetime
    df["datetime"] = pd.to_datetime(df["date"], utc=True)
    df.drop(columns=['date'], inplace=True)
    print("completed step 03: converted date column to datetime")

    # create language column
    df["language"] = df["text"].apply(detect_language)
    print("completed step 04: created language column")

    # translate to english
    df["text_en"] = df.apply(
        lambda row: translate_to_english(row["text"], row["language"]),
        axis=1
    )
    print("completed step 05: translated to english\n")
    print("data preprocessing is completed...")

    return df.reset_index(drop=True)

In [44]:
df = preprocess_pipeline("/Users/danukalakshan/Projects/review-analysis/data/trustpilot/task_rabbit.csv")

data preprocessing is started...

completed step 01: dropped rows with null values
completed step 02: dropped records of reviewers who has more than 5 reviews
completed step 03: converted date column to datetime
completed step 04: created language column
completed step 05: translated to english

data preprocessing is completed...


In [45]:
df.shape

(111, 6)

In [47]:
df.head()

,reviewer,text,stars,datetime,language,text_en
0,"John, Morgan",Norbert is great.,5,2025-11-26 17:29:50+00:00,no,Norbert is great.
1,KM,"Great experience. Very prompt, reliable and fast.",5,2025-10-26 14:48:22+00:00,en,"Great experience. Very prompt, reliable and fast."
2,MR R E BRANT,Paid Ikea to have wardrobes built via task rab...,1,2025-10-16 12:27:03+00:00,it,Paid Ikea to have wardrobes built via task rab...
3,Divya Pavan,The person was so experienced and he did the e...,5,2025-10-13 09:40:58+00:00,en,The person was so experienced and he did the e...
4,Ilaria Rodella,Puntualità ed efficienza,4,2025-07-31 10:49:07+00:00,it,Punctuality and efficiency


In [64]:
# Try out transfomer model

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

def classify_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return None
    result = sentiment_pipeline(text[:512])[0]  # limit length
    return result["label"], result["score"]

Device set to use mps:0


In [65]:
df[["sentiment", "sentiment_confidence"]] = df["text_en"].apply(
    lambda x: pd.Series(classify_sentiment(x))
)

In [66]:
label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}

df["sentiment"] = df["sentiment"].map(label_map)

In [ ]:
def classify_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return None
    result = sentiment_pipeline(text[:512])[0]  # limit length
    return result["label"], result["score"]

def sentiment_analysis(data_path):
    df = pd.read_csv(data_path)

    # schema check
    required_columns = {"text_en"}
    missing = required_columns - set(df.columns)
    if missing:
        raise ValueError(f"Invalid data structure. Missing columns: {missing}")
    print("sentiment analysis is started...\n")

    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment"
    )
    print("sentiment pipeline is loaded.")

    df[["sentiment", "sentiment_confidence"]] = df["text_en"].apply(
        lambda x: pd.Series(classify_sentiment(x))
    )

    label_map = {
        "LABEL_0": "negative",
        "LABEL_1": "neutral",
        "LABEL_2": "positive"
    }
    
    df["sentiment"] = df["sentiment"].map(label_map)
    print("\nsentiment analysis is finished...")

    return df

In [69]:
df.sample(10)

,reviewer,text,stars,datetime,language,text_en,sentiment,sentiment_confidence
64,Francesca,Persona puntuale ed efficiente,4,2024-06-06 08:49:53+00:00,it,Punctual and efficient person,positive,0.557328
70,Jessica,"Veloci, educati e super preparati!",5,2024-05-16 08:05:23+00:00,it,"Fast, polite and super prepared!",positive,0.866556
92,Aleksandra Mikhailichenko,Professionalità. Lavorano veloce e non lascian...,5,2024-02-17 19:53:45+00:00,it,Professionalism. They work fast and leave no t...,positive,0.557164
52,CURTIS ELIZABETH,Was a very easy service to use and Stefano did...,5,2024-08-01 10:58:24+00:00,en,Was a very easy service to use and Stefano did...,positive,0.985239
99,Anna Rosanova,Mohamed e Ahmed sono stati efficienti e puntua...,5,2023-11-27 18:14:35+00:00,it,"Mohamed and Ahmed were efficient and punctual,...",positive,0.828505
3,Divya Pavan,The person was so experienced and he did the e...,5,2025-10-13 09:40:58+00:00,en,The person was so experienced and he did the e...,positive,0.989191
39,Marco Lopez,"Great service, high quality, fast and helpful !",5,2024-10-21 16:46:45+00:00,en,"Great service, high quality, fast and helpful !",positive,0.974734
32,Lisa Maria Pippus,Mohammed did a fabulous job with my Ikea Order...,5,2024-12-18 15:51:58+00:00,en,Mohammed did a fabulous job with my Ikea Order...,positive,0.985009
55,Vladimir,"Igor Zubcu is very fast, efficient, great prof...",5,2024-07-20 12:54:31+00:00,en,"Igor Zubcu is very fast, efficient, great prof...",positive,0.961588
14,Eugenia,"Efficienti, di parola",5,2025-04-08 13:36:32+00:00,it,"Efficient, true to their word",positive,0.575420


In [71]:
sentiment_analysis()